In [1]:
#流程
#构建计算图-LSTM模型:
#    embedding
#    LSTM
#    fc
#    train_op
#训练流程代码
#数据集代码
#    api：next_batch(batch_size)
#词表封装
#    api：sentence2id(sentence)句子转化id
# 类别的封装
#    api: category2id(text_category).
import tensorflow as tf
import os
import sys
import numpy as np
import math

tf.logging.set_verbosity(tf.logging.INFO)#在notebook中打印日志

E:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def get_default_params():#定义LSTM模型需要的参数
    return tf.contrib.training.HParams(#这个api用于管理参数，返回一个对象
        # for fully run
        # num_embedding_size = 32,
        # num_timesteps = 600,
        # num_lstm_nodes = [128, 128],
        # num_lstm_layers = 2,
        # num_fc_nodes = 128,
        num_embedding_size = 16,
        num_timesteps = 50,#对齐mini_batch,指定步长,数据在mini_batch里被截断
        num_filters = 64,#卷积核数目大小
        num_kernel_size = 3,
        num_fc_nodes = 32,#fc的神经单元数
        batch_size = 100,
        learning_rate = 0.001,
        num_word_threshold = 10,#忽略掉出现次数不够的词语
    )

hps = get_default_params()

train_file = '.\\cnews_data\\cnews.train.seg.txt'
val_file = '.\\cnews_data\\cnews.val.seg.txt'
test_file = '.\\cnews_data\\cnews.test.seg.txt'
vocab_file = '.\\cnews_data\\cnews.vocab.txt'
category_file = '.\\cnews_data\\cnews.category.txt'
output_folder = '.\\run_text_rnn'

if not os.path.exists(output_folder):
    os.mkdir(output_folder)

In [3]:
class Vocab:#词表封装模块
    def __init__(self, filename, num_word_threshold):
        self._word_to_id = {}
        self._unk = -1
        self._num_word_threshold = num_word_threshold
        self._read_dict(filename)
    
    def _read_dict(self, filename):
        with open(filename, 'r',encoding='utf-8') as f:
            lines = f.readlines()
        for line in lines:
            word, frequency = line.strip('\r\n').split('\t')
            frequency = int(frequency)
            if frequency < self._num_word_threshold:
                continue
            idx = len(self._word_to_id)
            if word == '<UNK>':
                self._unk = idx
            self._word_to_id[word] = idx
    
    def word_to_id(self, word):
        return self._word_to_id.get(word, self._unk)#词语不存在就返回unk的id
    
    @property#成员函数
    def unk(self):
        return self._unk
    
    def size(self):
        return len(self._word_to_id)
    
    def sentence_to_id(self, sentence):#把句子转化成一系列id
        word_ids = [self.word_to_id(cur_word) \
                    for cur_word in sentence.split()]
        return word_ids


class CategoryDict:
    def __init__(self, filename):
        self._category_to_id = {}
        with open(filename, 'r',encoding='utf-8') as f:
            lines = f.readlines()
        for line in lines:
            category = line.strip('\r\n')
            idx = len(self._category_to_id)
            self._category_to_id[category] = idx
    
    def size(self):
        return len(self._category_to_id)
        
    def category_to_id(self, category):
        if not category in self._category_to_id:
            raise Exception(
                "%s is not in our category list" % category)
        return self._category_to_id[category]
        
vocab = Vocab(vocab_file, hps.num_word_threshold)
vocab_size = vocab.size()
tf.logging.info('vocab_size: %d' % vocab_size)

category_vocab = CategoryDict(category_file)
num_classes = category_vocab.size()
tf.logging.info('num_classes: %d' % num_classes)

test_str = '时尚'
tf.logging.info(
    'label: %s, id: %d' % (
        test_str,
        category_vocab.category_to_id(test_str)))

INFO:tensorflow:vocab_size: 77323
INFO:tensorflow:num_classes: 10
INFO:tensorflow:label: 时尚, id: 5


In [4]:
class TextDataSet:
    def __init__(self, filename, vocab, category_vocab, num_timesteps):#输入文件名，词典，类别词典，对齐限制数
        self._vocab = vocab
        self._category_vocab = category_vocab
        self._num_timesteps = num_timesteps
        # matrix
        self._inputs = []#输入的文本,为矩阵
        # vector
        self._outputs = []#输出的类别值，为向量
        self._indicator = 0#指明当前batch已经到了数据的哪个位置
        self._parse_file(filename)
    
    def _parse_file(self, filename):
        tf.logging.info('Loading data from %s', filename)
        with open(filename, 'r',encoding='utf-8') as f:
            lines = f.readlines()
        for line in lines:
            label, content = line.strip('\r\n').split('\t')
            id_label = self._category_vocab.category_to_id(label)#把label值换成id
            id_words = self._vocab.sentence_to_id(content)#把句子换成id
            id_words = id_words[0: self._num_timesteps]#对行进行截断，如果过长
            padding_num = self._num_timesteps - len(id_words)#计算需要填充的量
            id_words = id_words + [#对行进行填充，如果不够长，填充unk
                self._vocab.unk for i in range(padding_num)]
            self._inputs.append(id_words)#加入输入
            self._outputs.append(id_label)#加入输出
        self._inputs = np.asarray(self._inputs, dtype = np.int32)#转成numpy的矩阵
        self._outputs = np.asarray(self._outputs, dtype = np.int32)#转成numpy的矩阵
        self._random_shuffle()#随机化函数
        self._num_examples = len(self._inputs)
    
    def _random_shuffle(self):
        p = np.random.permutation(len(self._inputs))
        self._inputs = self._inputs[p]
        self._outputs = self._outputs[p]
    
    def num_examples(self):
        return self._num_examples
    
    def next_batch(self, batch_size):
        end_indicator = self._indicator + batch_size
        if end_indicator > len(self._inputs):
            self._random_shuffle()
            self._indicator = 0
            end_indicator = batch_size
        if end_indicator > len(self._inputs):
            raise Execption("batch_size: %d is too large" % batch_size)
        
        batch_inputs = self._inputs[self._indicator: end_indicator]
        batch_outputs = self._outputs[self._indicator: end_indicator]
        self._indicator = end_indicator
        return batch_inputs, batch_outputs
            
train_dataset = TextDataSet(
    train_file, vocab, category_vocab, hps.num_timesteps) 
val_dataset = TextDataSet(
    val_file, vocab, category_vocab, hps.num_timesteps)
test_dataset = TextDataSet(
    test_file, vocab, category_vocab, hps.num_timesteps)

print(train_dataset.num_examples())
print(val_dataset.num_examples())
print(test_dataset.num_examples())

print(train_dataset.next_batch(2))
print(val_dataset.next_batch(2))
print(test_dataset.next_batch(2))

INFO:tensorflow:Loading data from .\cnews_data\cnews.train.seg.txt
INFO:tensorflow:Loading data from .\cnews_data\cnews.val.seg.txt
INFO:tensorflow:Loading data from .\cnews_data\cnews.test.seg.txt
50000
5000
10000
(array([[19188,  1481,   806,  6047, 60261,  6282, 25667,   760,  2629,
           17,   250,    16,   155,  2046,  4399,    11,    71,    15,
        41591,   178,  6346,    14,   646, 40307,     2,  1165,   493,
          244,     1, 34449,  1068,   389,  2091,     1,   161,   398,
        22445, 22555,  2629,     2,   760,  9242,     3,    60,  5625,
         5513,     2,  1958,   710,   322],
       [ 5310,    15, 24041,    14,     8, 30339,     9,  3604,    32,
        11119,   156,   481,   456,    82,    78,    75,    23,    75,
           44,   269,     1,   160,  5635,   270,     1,  3604,   126,
        21938,    56,  1897,     1,     0,  2539,  6842,  1887,     1,
         5365,   326,  6444,     2,  8552,    15,  3918,   407,   825,
           14,   159,   223,  

In [5]:
#构建计算图-LSTM模型:
#    embedding
#    LSTM
#    fc
#    train_op
def create_model(hps, vocab_size, num_classes):
    num_timesteps = hps.num_timesteps
    batch_size = hps.batch_size
    
    inputs = tf.placeholder(tf.int32, (batch_size, num_timesteps))
    outputs = tf.placeholder(tf.int32, (batch_size, ))
    keep_prob = tf.placeholder(tf.float32, name = 'keep_prob')#用于dropout
    
    global_step = tf.Variable(#保存步数
        tf.zeros([], tf.int64), name = 'global_step', trainable=False)
    
    embedding_initializer = tf.random_uniform_initializer(-1.0, 1.0)
    with tf.variable_scope('embedding', initializer = embedding_initializer):
        embeddings = tf.get_variable(#如果变量存在就重用，不存在就创建一个
            'embedding',
            [vocab_size, hps.num_embedding_size],
            tf.float32)
        # [1, 10, 7] -> [embeddings[1], embeddings[10], embeddings[7]]
        embed_inputs = tf.nn.embedding_lookup(embeddings, inputs)#把inputs转化成embedding
        
        
        
        
    scale = 1.0 / math.sqrt(hps.num_embedding_size + hps.num_filters) / 3.0
    cnn_init = tf.random_uniform_initializer(-scale, scale)
    with tf.variable_scope('cnn', initializer = cnn_init):
        # embed_inputs: [batch_size, timesteps, embed_size]
        # conv1d: [batch_size, timesteps, num_filters]
        conv1d = tf.layers.conv1d(embed_inputs,hps.num_filters,hps.num_kernel_size,activation = tf.nn.relu,)
        #输入，卷积核数目，卷积核大小，激活函数
        global_maxpooling = tf.reduce_max(conv1d, axis=[1])   # 
    
        
        
        
        
        
    
    fc_init = tf.uniform_unit_scaling_initializer(factor=1.0)
    with tf.variable_scope('fc', initializer = fc_init):
        fc1 = tf.layers.dense(global_maxpooling, 
                              hps.num_fc_nodes,
                              activation = tf.nn.relu,
                              name = 'fc1')
        fc1_dropout = tf.contrib.layers.dropout(fc1, keep_prob)
        logits = tf.layers.dense(fc1_dropout,
                                 num_classes,
                                 name = 'fc2')
    
    with tf.name_scope('metrics'):
        softmax_loss = tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits = logits, labels = outputs)#1.把labels做onehot，2.对logits计算概率，3.算loss
        loss = tf.reduce_mean(softmax_loss)
        # [0, 1, 5, 4, 2] -> argmax: 2
        y_pred = tf.argmax(tf.nn.softmax(logits),1, output_type = tf.int32)#logits=10*1，在列方向看谁最大
        correct_pred = tf.equal(outputs, y_pred)
        accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
    
    with tf.name_scope('train_op'):
        train_op = tf.train.AdamOptimizer(hps.learning_rate).minimize(loss, global_step=global_step)
    
    return ((inputs, outputs, keep_prob),
            (loss, accuracy),
            (train_op, global_step))

placeholders, metrics, others = create_model(
    hps, vocab_size, num_classes)

inputs, outputs, keep_prob = placeholders
loss, accuracy = metrics
train_op, global_step = others

Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.


In [6]:
def eval_holdout(sess, accuracy, dataset_for_test, batch_size):
    num_batches = dataset_for_test.num_examples() // batch_size
    tf.logging.info("Eval holdout: num_examples = %d, batch_size = %d",
                    dataset_for_test.num_examples(), batch_size)
    accuracy_vals = []
    for i in range(num_batches):
        batch_inputs, batch_labels = dataset_for_test.next_batch(batch_size)
        accuracy_val = sess.run(accuracy,
                                feed_dict = {
                                    inputs: batch_inputs,
                                    outputs: batch_labels,
                                    keep_prob: 1.0,
                                })
        accuracy_vals.append(accuracy_val)
    return np.mean(accuracy_vals)

In [7]:
init_op = tf.global_variables_initializer()
train_keep_prob_value = 0.8
test_keep_prob_value = 1.0
num_train_steps = 10000

# Train: 99.7%
# Valid: 92.7%
# Test:  93.2%
with tf.Session() as sess:
    sess.run(init_op)
    for i in range(num_train_steps):
        batch_inputs, batch_labels = train_dataset.next_batch(hps.batch_size)
        outputs_val = sess.run([loss, accuracy, train_op, global_step],
                               feed_dict = {
                                   inputs: batch_inputs,
                                   outputs: batch_labels,
                                   keep_prob: train_keep_prob_value,
                               })
        loss_val, accuracy_val, _, global_step_val = outputs_val
        if global_step_val % 200 == 0:
            tf.logging.info("Step: %5d, loss: %3.3f, accuracy: %3.3f"
                            % (global_step_val, loss_val, accuracy_val))
        
        if global_step_val % 1000 == 0:
            accuracy_eval = eval_holdout(sess, accuracy, val_dataset, hps.batch_size)
            accuracy_test = eval_holdout(sess, accuracy, test_dataset, hps.batch_size)
            tf.logging.info("Step: %5d, val_accuracy: %3.3f, test_accuracy: %3.3f"
                            % (global_step_val, accuracy_eval, accuracy_test))

INFO:tensorflow:Step:   200, loss: 1.535, accuracy: 0.490
INFO:tensorflow:Step:   400, loss: 1.178, accuracy: 0.600
INFO:tensorflow:Step:   600, loss: 0.752, accuracy: 0.790
INFO:tensorflow:Step:   800, loss: 0.658, accuracy: 0.820
INFO:tensorflow:Step:  1000, loss: 0.324, accuracy: 0.920
INFO:tensorflow:Eval holdout: num_examples = 5000, batch_size = 100
INFO:tensorflow:Eval holdout: num_examples = 10000, batch_size = 100
INFO:tensorflow:Step:  1000, val_accuracy: 0.765, test_accuracy: 0.790


KeyboardInterrupt: 